<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [37]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [ ]:
#A={a1, a2, a3}
#countPos = 3

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

In [38]:
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [39]:
def x1(x, v):
  return math.sin(x[0]**2 + math.log(math.fabs(v[1] + v[0])) / x[2])

def x2(x, v):
  return math.cos(v[0] * x[2] - (x[1] / x[2])**3)

def x3(x, v):
  return math.sin(math.tan(x[2] * v[2]) - 3**(v[0]))

def v1(x, v):
  return (x[1] * v[0] * v[1] + math.tan(x[2])**2)

def v2(x, v):
  return (0.2 * v[0] + 33.0 * v[1] + v[2])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(math.fabs(x[1] * x[2])))

In [57]:
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================

def createVariants(function, elem = 100):
  vector = []
  for i in range(elem):
    vector.append(function(generationRandomVector(), generationRandomVector()))
  return vector

#=================================================================
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps):
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

In [58]:
table_t0 = pd.DataFrame()
#column = ['x1(0)', 'x2(0)', 'x3(0)', 'v1(0)', 'v2(0)', 'v3(0)']

table_t0['x1(0)'] = createVariants(x1)
table_t0['x2(0)'] = createVariants(x2)
table_t0['x3(0)'] = createVariants(x3)
table_t0['v1(0)'] = createVariants(v1)
table_t0['v2(0)'] = createVariants(v2)
table_t0['v3(0)'] = createVariants(v3)

print("Исходные данные при t = 0")
table_t0

Исходные данные при t = 0


,x1(0),x2(0),x3(0),v1(0),v2(0),v3(0)
0,-0.963198,0.880866,-0.891224,0.124644,32.833313,-0.168703
1,-0.784310,0.892975,0.508723,93.364412,-64.537256,3.600163
2,0.079106,0.799419,0.837536,1.555212,-27.228517,7.746124
3,0.614495,-0.786302,-0.424866,107.185056,33.258346,1.648622
4,0.998384,-0.668321,-0.955653,6.001484,38.587578,0.441130
...,...,...,...,...,...,...
95,0.636211,0.448437,-0.403549,3.997047,42.219574,0.886799
96,-0.054302,0.974690,-0.169408,0.534600,81.689529,-0.055516
97,-0.614203,-0.816255,-0.839444,-3.585675,77.913962,14.834146
98,-0.449198,-0.232423,-0.987255,-5.783908,-70.358816,0.585539


In [51]:
table_t1 = pd.DataFrame()
#column = ['x1(0)', 'x2(0)', 'x3(0)']

table_t1['x1(1)'] = createVariants(x1)
table_t1['x2(1)'] = createVariants(x2)
table_t1['x3(1)'] = createVariants(x3)

print("t = 1")
table_t1

t = 1


,x1(1),x2(1),x3(1)
0,-0.978926,0.613615,-0.051806
1,0.008825,0.933149,0.845445
2,0.571387,-0.158484,-0.620593
3,0.723878,0.997171,-0.462486
4,0.777195,-0.115033,0.066554
...,...,...,...
95,0.355012,-0.551402,0.599516
96,-0.949853,-0.912019,-0.663930
97,0.881984,-0.780736,-0.054600
98,-0.867813,0.638115,0.000586
